# Chicago Yelp API

### imports


In [21]:
import requests
import pandas as pd

### load api key

In [24]:
with open("api_key_yelp.txt", "r") as file:
    api_key = file.read().strip()

### header and params

In [27]:
headers = {"Authorization": f"Bearer {api_key}"}
url = "https://api.yelp.com/v3/businesses/search"

In [29]:
params = {"location": "Chicago", "term": "restaurants"}

### send req

In [32]:
response = requests.get(url, headers=headers, params=params)
data = response.json()

### get df

In [35]:
businesses = data.get("businesses", [])
df = pd.json_normalize(businesses)
df.head()

""


In [39]:
print(response.status_code)
print(data)

400
{'error': {'code': 'TRIAL_EXPIRED', 'description': 'Your Trial has expired. Please upgrade to continue using the service. Visit https://www.yelp.com/developers/v3/manage_app or email at fusion@yelp.com to explore options.'}}
